In [1]:
import itertools
import jsonlines

from datasets import load_dataset
from pprint import pprint

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [2]:
instruction_tuned_dataset = load_dataset("tatsu-lab/alpaca", split="train", streaming= True)

README.md:   0%|          | 0.00/7.47k [00:00<?, ?B/s]

In [4]:
m = 5 
print("Instruction-tuned dataset:")
top_m = list(itertools.islice(instruction_tuned_dataset,m))
for j in top_m:
    pprint(j)

Instruction-tuned dataset:
{'input': '',
 'instruction': 'Give three tips for staying healthy.',
 'output': '1.Eat a balanced diet and make sure to include plenty of fruits '
           'and vegetables. \n'
           '2. Exercise regularly to keep your body active and strong. \n'
           '3. Get enough sleep and maintain a consistent sleep schedule.',
 'text': 'Below is an instruction that describes a task. Write a response that '
         'appropriately completes the request.\n'
         '\n'
         '### Instruction:\n'
         'Give three tips for staying healthy.\n'
         '\n'
         '### Response:\n'
         '1.Eat a balanced diet and make sure to include plenty of fruits and '
         'vegetables. \n'
         '2. Exercise regularly to keep your body active and strong. \n'
         '3. Get enough sleep and maintain a consistent sleep schedule.'}
{'input': '',
 'instruction': 'What are the three primary colors?',
 'output': 'The three primary colors are red, blue, and

In [5]:
prompt_template_with_input = """Below is an instruction that describe a task, paired with an input that provides furthur content
### Instruction:
{instruction}

### Input: 
{input}

### Response:"""

prompt_template_without_input = """Below is an instruction that describe a task. Write a response that appropriately complete it.
### Instruction:
{instruction}

### Response:"""

In [9]:
procesed_data = []
for j in top_m:
    if not j["input"]:
        processed_prompt = prompt_template_without_input.format(instruction = j["instruction"])
    else:
        processed_prompt = prompt_template_with_input.format(instruction = j["instruction"], input = j["input"])
    procesed_data.append({"input": processed_prompt, "output": j["output"]})

In [10]:
pprint(procesed_data[0])

{'input': 'Below is an instruction that describe a task. Write a response that '
          'appropriately complete it.\n'
          '### Instruction:\n'
          'Give three tips for staying healthy.\n'
          '\n'
          '### Response:',
 'output': '1.Eat a balanced diet and make sure to include plenty of fruits '
           'and vegetables. \n'
           '2. Exercise regularly to keep your body active and strong. \n'
           '3. Get enough sleep and maintain a consistent sleep schedule.'}


In [11]:
with jsonlines.open(f'alpaca_processed.jsonl', 'w') as writer:
    writer.write_all(procesed_data)

In [12]:
dataset_path_hf = 'lamini/alpaca'
dataset_hf = load_dataset(dataset_path_hf)
print(dataset_hf)

README.md:   0%|          | 0.00/388 [00:00<?, ?B/s]

(…)-00000-of-00001-f98ba950981285f9.parquet:   0%|          | 0.00/12.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 52002
    })
})


In [19]:
# Use a pipeline as a high-level helper
from transformers import pipeline

messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe = pipeline("text-generation", model="Qwen/Qwen2.5-Coder-3B-Instruct")
pipe(messages)

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.21G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

Device set to use cpu


[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': "I am Qwen, an AI developed by Alibaba Cloud. I'm here to assist with answering questions"}]}]

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-Coder-3B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "write a quick sort algorithm."
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]